## Modelling

Here a quick overview what models we used:
- IsolationForest
- SVM
- LOV
- DBSCan
- XGBoost

The main question we want to solve using these models is:
<div class="alert alert-block alert-success">
Can we find any anomalies based on data we were given?
</div>

### IsolationForest